In [1]:
!pip install -q streamlit pyngrok transformers accelerate bitsandbytes
!npm install -g localtunnel
!pip install torch sqlparse
!pip install fastapi uvicorn pyngrok nest-asyncio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 34.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
SQL_MODEL_NAME = "khalifa1/Sql_LLama2_V3"
sql_tokenizer = AutoTokenizer.from_pretrained(SQL_MODEL_NAME)

# Check GPU memory
gpu_mem = torch.cuda.get_device_properties(0).total_memory if torch.cuda.is_available() else 0

if gpu_mem > 15e9:  # 15GB threshold
    sql_model = AutoModelForCausalLM.from_pretrained(
        SQL_MODEL_NAME,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True
    )
else:
    sql_model = AutoModelForCausalLM.from_pretrained(
        SQL_MODEL_NAME,
        load_in_4bit=True,
        device_map="auto",
        trust_remote_code=True
    )

print("Model loaded successfully!")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.93k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.62M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.59G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Model loaded successfully!


In [56]:
# AuthToken form ngrok
!ngrok authtoken 2tSXRwnaiezmuL4qx4PsL6mLhKQ_5df8bxbmWkfYicDkGKya5

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [57]:
from fastapi import FastAPI
from pydantic import BaseModel
from pyngrok import ngrok
import uvicorn
import torch
import sqlparse
import nest_asyncio
import threading
from fastapi.middleware.cors import CORSMiddleware

nest_asyncio.apply()

app = FastAPI()

# Add middleware
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["GET", "POST", "OPTIONS"],
    allow_headers=["*"],
)

# Define request model
class QueryRequest(BaseModel):
    question: str
    schema: str = ""  # Optional schema

# Define the SQL generation function
def generate_query(question, schema=""):
    # Define the base prompt with or without schema
    if schema:
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate a SQL query to answer this question: `{question}`

DDL statements:

{schema}
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The following SQL query best answers the question `{question}`:
```sql
"""
    else:
        prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Generate a SQL query to answer this question: `{question}`
<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The following SQL query best answers the question `{question}`:
```sql
"""

    # Tokenize input for the model
    inputs = sql_tokenizer(prompt, return_tensors="pt").to("cuda")

    # Generate response
    generated_ids = sql_model.generate(
        **inputs,
        num_return_sequences=1,
        eos_token_id=sql_tokenizer.eos_token_id,
        pad_token_id=sql_tokenizer.eos_token_id,
        max_new_tokens=400,
        do_sample=False,
        num_beams=1,
        temperature=0.0,
        top_p=1,
    )

    outputs = sql_tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

    # Clear GPU cache
    torch.cuda.empty_cache()
    torch.cuda.synchronize()

    # Extract SQL output and format
    raw_sql = outputs[0].split("```sql")[1].split(";")[0]
    formatted_sql = sqlparse.format(raw_sql, reindent=True).strip()

    return formatted_sql

# Home to test server
@app.get("/")
def home():
    return {"message": "Hello From Generate SQL code BOT"}

# Model Endpoint
@app.post("/generate_sql/")
async def generate_sql(request: QueryRequest):
    sql_query = generate_query(request.question, request.schema)
    return {"sql_query": sql_query}

# Function to run FastAPI with threading    (port 3000)
def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=3000)

# Start ngrok tunnel (port 3000)
ngrok_tunnel = ngrok.connect(3000)
print(f"Public URL: {ngrok_tunnel.public_url}") # ngrok Url will print here and can use it to access endpoint

# Run FastAPI in a separate thread
fastapi_thread = threading.Thread(target=run_fastapi, daemon=True)
fastapi_thread.start()

# Keep the main thread alive
try:
    fastapi_thread.join()
except KeyboardInterrupt:
    print("Shutting down server...")

Public URL: https://70d9-34-143-189-62.ngrok-free.app


INFO:     Started server process [218]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 3000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.


**Test Endpoint in Python**

In [ ]:
import requests

# Set your FastAPI URL (replace with your ngrok link)
API_URL = "URL"

# Test Case 1: Only a Question
payload_1 = {
    "question": "Get all products priced above 100"
}

# Test Case 2: Question with Schema
payload_2 = {
    "question": "List customers from New York",
    "schema": """CREATE TABLE customers (
                    id INT PRIMARY KEY,
                    name VARCHAR(50),
                    address VARCHAR(100)
                 );"""
}

# Send Request for Test Case 1
response_1 = requests.post(API_URL, json=payload_1)
print("Test 1 - Only Question:\n", response_1.json())

# Send Request for Test Case 2
response_2 = requests.post(API_URL, json=payload_2)
print("\nTest 2 - Question with Schema:\n", response_2.json())


**Kill ngrok**

In [58]:
!pkill uvicorn
!pkill ngrok